In [129]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import mysql.connector as connector



%reload_ext dotenv
%dotenv
import os
# load the database password from an .env file
db_pass = os.getenv("DB_PASS")

dbconfig = {
    "database":"little_lemon_db",
    "user" : "root",
    "password" : db_pass
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_b",
                           pool_size = 2,
                           **dbconfig)
    print("***The connection pool is created with a name: ",pool.pool_name)
    print("***The pool size is:",pool.pool_size)
except:
    print("***No more connections are available.")
    print("***Adding new connection in the pool.")
    
    # Create a connection
    connection=connector.connect(user="root",password=db_pass)
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    print("***A new connection is added in the pool.\n")
    
    user_connected = pool.get_connection()


print("***Getting a connection from the pool.")
connection1 = pool.get_connection()

cursor = connection1.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS little_lemon_db") 
print("**Database created")
cursor.execute("USE little_lemon_db")
if connection1.is_connected():
    cursor.close()
    print("***The cursor is closed.")
    connection1.close()
    print("***MySQL connection is closed.")
else:
    print("***Connection is already closed")

***The connection pool is created with a name:  pool_b
***The pool size is: 2
***Getting a connection from the pool.
**Database created
***The cursor is closed.
***MySQL connection is closed.


In [130]:
# Task 2
## Guest 1
connection1 = pool.get_connection()
cursor = connection1.cursor()

insert_bookings="""
REPLACE INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(7, 8, 'Anees','Java','18:00:00',6);"""

# Populate Bookings table
cursor.execute(insert_bookings)
connection1.commit()
print("**Booking inserted")

## Guest 2
connection2 = pool.get_connection()
cursor = connection1.cursor()
insert_bookings="""
REPLACE INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(8, 5, 'Bald', 'Vin', '19:00:00', 6);"""

# Populate Bookings table
cursor.execute(insert_bookings)
connection2.commit()
print("**Booking inserted")


## Guest 3
connection=connector.connect(user="root",password=db_pass)
# Add the connection into the pool
pool.add_connection(cnx=connection)
print("***A new connection is added in the pool.\n")

connection3 = pool.get_connection()
cursor = connection1.cursor()
insert_bookings="""
REPLACE INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(9, 12, 'Jay', 'Kon', '19:30:00', 6);"""

# Populate Bookings table
cursor.execute(insert_bookings)
connection3.commit()
print("**Booking inserted")

# Return connection
for connection in [connection1,connection2]:
    if connection.is_connected():
        cursor.close()
        print("***The cursor is closed.")
        connection.close()
        print("***MySQL connection is closed.")
    else:
        print("***Connection is already closed")

**Booking inserted
**Booking inserted
***A new connection is added in the pool.

**Booking inserted
***The cursor is closed.
***MySQL connection is closed.
***The cursor is closed.
***MySQL connection is closed.


In [131]:
# Task3
print("***Getting a connection from the pool.")
connection1 = pool.get_connection()
cursor = connection1.cursor()

# Query to retrieve all bookings is: 
manager_query = """SELECT Name, EmployeeID FROM Employees WHERE ROLE = "Manager";"""

# Execute query 
cursor.execute(manager_query)

# Fetch fist 3 records in results
results = cursor.fetchmany(size=3)
cursor.close()

# Retrieve column names
cols = cursor.column_names

# Print column names and records from results using for loop
print("Data retrieved:")
print(cols)
manager = results[0]
for result in results:
    print(result)

***Getting a connection from the pool.
Data retrieved:
('Name', 'EmployeeID')
('Mario Gollini', 1)


In [132]:
# Task3
# Query to retrieve all bookings is: 
sal_query = """SELECT Name, Role FROM Employees ORDER BY Annual_Salary DESC LIMIT 1;"""

# Execute query 
cursor = connection1.cursor()
cursor.execute(sal_query)

# Fetch fist 3 records in results
results = cursor.fetchmany(size=3)
cursor.close()

# Retrieve column names
cols = cursor.column_names

# Print column names and records from results using for loop
print("Data retrieved:")
print(cols)
highest_salary = results[0]
for result in results:
    print(result)

Data retrieved:
('Name', 'Role')
('Mario Gollini', 'Manager')


In [133]:
# Task3
# Query to retrieve all bookings is: 
guest_query = """SELECT COUNT(*) FROM Bookings WHERE HOUR(BookingSlot) BETWEEN 18 AND 20;"""

# Execute query 
cursor = connection1.cursor()
cursor.execute(guest_query)

# Fetch fist 3 records in results
results = cursor.fetchall()
cursor.close()

# Retrieve column names
cols = cursor.column_names

# Print column names and records from results using for loop
print("Data retrieved:")
print(cols)
num_guest = results[0]
for result in results:
    print(result)

Data retrieved:
('COUNT(*)',)
(7,)


In [134]:
# Task3
# Query to retrieve all bookings is: 
waiting_query = """
SELECT CONCAT(GuestFirstName, ' ', GuestLastName), 
BookingID 
FROM Bookings
WHERE EmployeeID = 6
ORDER BY BookingSlot;"""

# Execute query 
cursor = connection1.cursor()
cursor.execute(waiting_query)

# Fetch fist 3 records in results
results = cursor.fetchall()
cursor.close()

# Retrieve column names
cols = cursor.column_names

# Print column names and records from results using for loop
print("Data retrieved:")
waiting = results
print(cols)
for result in results:
    print(result)

Data retrieved:
("CONCAT(GuestFirstName, ' ', GuestLastName)", 'BookingID')
('Anees Java', 7)
('Bald Vin', 8)
('Jay Kon', 9)


In [135]:
# Report task 3
print("Report:")
print(f"Manager: {manager[0]}, Id: {manager[1]}")
print(f"Highest salary: Name: {highest_salary[0]}, Role: {highest_salary[1]}")
print(f"Number of guest booked between 18 and 20: {num_guest[0]}")
print(f"Waiting Guest:")
for i in range(3):
    print(f"\tName: {waiting[i][0]}, Bookingslot: {waiting[i][1]}")

Report:
Manager: Mario Gollini, Id: 1
Highest salary: Name: Mario Gollini, Role: Manager
Number of guest booked between 18 and 20: 7
Waiting Guest:
	Name: Anees Java, Bookingslot: 7
	Name: Bald Vin, Bookingslot: 8
	Name: Jay Kon, Bookingslot: 9


In [136]:
# Task4
# Query to retrieve all bookings is: 
stored_procedure_query="""
CREATE PROCEDURE IF NOT EXISTS BasicSalesReport()

BEGIN

SELECT SUM(BillAmount), AVG(BillAmount), MIN(BillAmount), Max(BillAmount)
FROM Orders;

END

"""

# Execute the query
cursor = connection1.cursor()
cursor.execute(stored_procedure_query)

#********************************************#

# Call the stored procedure with its name
cursor.callproc("BasicSalesReport")
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()
cursor.close()
# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]



# Print data 
for data in dataset:
    print(f"Sales report: Total: {data[0]}, Average: {round(data[1],2)}, Min: {data[2]}, Max: {data[3]},")

if connection1.is_connected():
    cursor.close()
    print("***The cursor is closed.")
    connection1.close()
    print("***MySQL connection is closed.")
else:
    print("***Connection is already closed")


Sales report: Total: 243, Average: 48.60, Min: 37, Max: 86,
***The cursor is closed.
***MySQL connection is closed.


In [137]:
# Task 5
print("***Getting connection")
connection2 = pool.get_connection()

cursor = connection2.cursor(buffered = True)
print("***Cursor created")


query = """
SELECT BookingSlot, CONCAT(GuestFirstName, ' ', GuestLastName), Name, Role
FROM Bookings LEFT JOIN Employees  
ON Bookings.EmployeeID = Employees.EmployeeID
ORDER BY BookingSlot ASC
LIMIT 3;
"""

cursor.execute(query)

results = cursor.fetchall()
cursor.close()

# Retrieve column names
cols = cursor.column_names

# Print column names and records from results using for loop
print("""\nKitchen Screen:""")
for result in results:
    print(f"BookingSlot: {result[0]}")
    print(f"\tGuest Name: {result[1]}")
    print(f"\tAssigned to: {result[2]} [{result[3]}]")
    print("\n")

# Closing connection
if connection2.is_connected():
    cursor.close()
    print("***The cursor is closed.")
    connection2.close()
    print("***MySQL connection is closed.")
else:
    print("***Connection is already closed")

***Getting connection
***Cursor created

Kitchen Screen:
BookingSlot: 15:00:00
	Guest Name: Vanessa McCarthy
	Assigned to: Giorgos Dioudis [Head Chef]


BookingSlot: 17:30:00
	Guest Name: Marcos Romero
	Assigned to: Fatma Kaya [Assistant Chef]


BookingSlot: 18:00:00
	Guest Name: Anees Java
	Assigned to: John Millar [Receptionist]


***The cursor is closed.
***MySQL connection is closed.
